In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
data = pd.read_csv('/home/l/Downloads/digit-recognizer/train.csv')

In [5]:
data.head(20)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
Y = data['label']
X = data.drop(['label'],axis=1)

m,n = X.shape

Y = np.array(Y)
X = np.array(X)
np.random.shuffle(X)

X = X.T
X.shape

(784, 42000)

In [101]:
W1 = np.random.rand(10, n) - 0.5
B1 = np.random.rand(10,1) - 0.5
W2 = np.random.rand(10,10) - 0.5
B2 = np.random.rand(10,1) - .5
W3 = np.random.rand(10,10) - .5
B3 = np.random.rand(10,1) - .5

In [102]:
def relu(x):
    return np.maximum(x,0)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=0, keepdims=True)) 
    return exp_x / np.sum(exp_x, axis=0, keepdims=True)  
    
def deriv_relu(x):
    return (x > 0).astype(int)

In [103]:
def train_test_split(x, y, test_size=0.2):
    num = int((1 - test_size) * m)
    x_train = x[:, :num]
    y_train = y[:num]
    x_test = x[:, num:]
    y_test = y[num:]
    
    return x_train, y_train, x_test, y_test

def one_hot_enc(y, classes=10):
    one_hot = np.zeros((y.size, classes))
    one_hot[np.arange(y.size), y] = 1
    
    return one_hot

In [104]:
def fwd_prop(x, W1, B1, W2, B2, W3, B3):
    
    Z1 = np.dot(W1,x) + B1
    A1 = relu(Z1)
    Z2 = np.dot(W2,A1) + B2
    A2 = relu(Z2)
    Z3 = np.dot(W3,A2) + B3
    A3 = softmax(Z3)
    
    return Z1, A1, Z2, A2, Z3, A3


def back_prop(x, y, Z1, A1, Z2, A2, Z3, A3, b_s=1):
    dZ3 = A3 - y
    dW3 = np.dot(dZ3,A2.T) / b_s 
    dB3 = np.sum(dZ3, axis=1, keepdims=True) / b_s
    dA2 = np.dot(W3,dZ3)
    dZ2 = dA2 * deriv_relu(Z2) 
    dW2 = np.dot(dZ2, A1.T) / b_s
    dB2 = np.sum(dZ2, axis=1, keepdims=True) / b_s

    dA1 = np.dot(W2, dZ2)
    dZ1 = dA1 * deriv_relu(Z1)
    dW1 = np.dot(dZ1, x.T) / b_s
    dB1 = np.sum(dZ1, axis=1, keepdims=True) / b_s
     
    return dW3, dB3, dW2, dB2, dW1, dB1

In [105]:
B1.shape

(10, 1)

In [109]:
X_train, Y_train, X_test, Y_test = train_test_split(X, Y)
batch_size = 10
eta = 0.01
eta_decay = 0.99

In [111]:
for epoch in range(400):
    for i in range(0, X_train.shape[1], batch_size):
       
        X_batch = X_train[:,i:i+batch_size]
        Y_target = one_hot_enc(Y_train[i:i+batch_size]).T
    
    
        Z1, A1, Z2, A2, Z3, A3 = fwd_prop(X_batch, W1, B1, W2, B2, W3, B3)
        dW3, dB3, dW2, dB2, dW1, dB1 = back_prop(X_batch, Y_target, Z1, A1, Z2, A2, Z3, A3, b_s=10)

        loss = -np.mean(np.sum(Y_target * np.log(A3 + 1e-8)))/10

        W1 -= eta*dW1
        W2 -= eta*dW2
        W3 -= eta*dW3
        B1 -= eta*dB1
        B2 -= eta*dB2
        B3 -= eta*dB3

    if epoch%100 == 0 :
        print(f'Epoch {epoch+1}/{100}, Loss: {loss:.4f}, {A3}, {Y_target}')
    eta *= eta_decay
        

Epoch 1/100, Loss: 2.3204, [[0.09759923 0.09759923 0.09759923 0.09759923 0.09759923 0.09759923
  0.09759923 0.09759923 0.09759923 0.09759923]
 [0.11114514 0.11114514 0.11114514 0.11114514 0.11114514 0.11114514
  0.11114514 0.11114514 0.11114514 0.11114514]
 [0.10046003 0.10046003 0.10046003 0.10046003 0.10046003 0.10046003
  0.10046003 0.10046003 0.10046003 0.10046003]
 [0.10324629 0.10324629 0.10324629 0.10324629 0.10324629 0.10324629
  0.10324629 0.10324629 0.10324629 0.10324629]
 [0.09705975 0.09705975 0.09705975 0.09705975 0.09705975 0.09705975
  0.09705975 0.09705975 0.09705975 0.09705975]
 [0.09071025 0.09071025 0.09071025 0.09071025 0.09071025 0.09071025
  0.09071025 0.09071025 0.09071025 0.09071025]
 [0.09839525 0.09839525 0.09839525 0.09839525 0.09839525 0.09839525
  0.09839525 0.09839525 0.09839525 0.09839525]
 [0.10466851 0.10466851 0.10466851 0.10466851 0.10466851 0.10466851
  0.10466851 0.10466851 0.10466851 0.10466851]
 [0.09706049 0.09706049 0.09706049 0.09706049 0.09706

KeyboardInterrupt: 